In [1]:
import pandas as pd


In [7]:
df = pd.read_csv('../data/data.csv')
data_artist_df = pd.read_csv('../data/data_by_artist.csv')
data_genre_df = pd.read_csv('../data/data_by_genres.csv')
data_year_df = pd.read_csv('../data/data_by_year.csv')
data_w_genre_df = pd.read_csv('../data/data_w_genres.csv')

In [8]:
df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920


In [9]:
data_artist_df.shape

(32539, 15)

In [10]:
df['artists'] = df['artists'].str.strip('[]')

In [11]:
df['artists'] = df['artists'].str.strip('"')

In [12]:
df['artists'] = df['artists'].str.strip("'")

In [13]:
df["artists"]

0                            Mamie Smith
1                  Screamin' Jay Hawkins
2                            Mamie Smith
3                        Oscar Velazquez
4                                   Mixe
                       ...              
174384    DJ Combo', 'Sander-7', 'Tony T
174385                      Alessia Cara
174386                         Roger Fly
174387                      Taylor Swift
174388                         Roger Fly
Name: artists, Length: 174389, dtype: object

In [14]:
df = df.drop(columns='release_date')


In [15]:
df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year
0,0.991000,Mamie Smith,0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,0.0936,149.976,0.6340,1920
1,0.643000,Screamin' Jay Hawkins,0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,0.0534,86.889,0.9500,1920
2,0.993000,Mamie Smith,0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,0.1740,97.600,0.6890,1920
3,0.000173,Oscar Velazquez,0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,0.0425,127.997,0.0422,1920
4,0.295000,Mixe,0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,0.0768,122.076,0.2990,1920


In [16]:
number_cols = ['acousticness', 'danceability', 'duration_ms', 'energy', 'explicit', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
               'popularity', 'speachiness', 'tempo', 'valence', 'year']

In [17]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

cluster_pipeline = Pipeline([('scaler', StandardScaler()),('kmeans',
                                                           KMeans(n_clusters=10, n_jobs=-1))])
X = data_genre_df.select_dtypes(np.number)
cluster_pipeline.fit(X)
data_genre_df['cluster'] = cluster_pipeline.predict(X)

In [18]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne_pipeline = Pipeline([('scaler', StandardScaler()), ('tsne', TSNE(n_components=2, verbose=2))])

genre_embedding = tsne_pipeline.fit_transform(X)

projection = pd.DataFrame(columns=['x', 'y'], data=genre_embedding)
projection['genres'] = data_genre_df['genres']
projection['cluster'] = data_genre_df['cluster']

fig = px.scatter(
    projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'genres'])
fig.show()


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 3232 samples in 0.006s...
[t-SNE] Computed neighbors for 3232 samples in 0.311s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3232
[t-SNE] Computed conditional probabilities for sample 2000 / 3232
[t-SNE] Computed conditional probabilities for sample 3000 / 3232
[t-SNE] Computed conditional probabilities for sample 3232 / 3232
[t-SNE] Mean sigma: 0.789973
[t-SNE] Computed conditional probabilities in 0.094s
[t-SNE] Iteration 50: error = 82.8634796, gradient norm = 0.0280992 (50 iterations in 0.892s)
[t-SNE] Iteration 100: error = 76.4002304, gradient norm = 0.0034188 (50 iterations in 0.813s)
[t-SNE] Iteration 150: error = 76.1978455, gradient norm = 0.0003526 (50 iterations in 0.758s)
[t-SNE] Iteration 200: error = 76.1820526, gradient norm = 0.0002235 (50 iterations in 0.773s)
[t-SNE] Iteration 250: error = 76.1791382, gradient norm = 0.0001826 (50 iterations in 0.751s)
[t-SNE] KL divergence after 250 iterati

In [19]:
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()),
                                  ('kmeans', KMeans(n_clusters=20,
                                                    verbose=2, n_jobs=-1))], verbose=True)
X = df.select_dtypes(np.number)
number_cols = list(X.columns)
song_cluster_pipeline.fit(X)
song_cluster_labels = song_cluster_pipeline.predict(X)
df['cluster_label'] = song_cluster_labels

inertia 1129836.4380921596
Iteration 17, inertia 1128466.3963794902
Iteration 18, inertia 1127713.8936848799
Iteration 19, inertia 1127250.8925300597
Iteration 20, inertia 1126891.7608000084
Iteration 21, inertia 1126706.785007255
Iteration 22, inertia 1126555.9870608156
Iteration 23, inertia 1126365.220545629
Iteration 24, inertia 1126187.67550452
Iteration 25, inertia 1125963.8333912077
Iteration 26, inertia 1125798.512254445
Iteration 27, inertia 1125662.2623232212
Iteration 28, inertia 1125511.8186746691
Iteration 29, inertia 1125367.1201407507
Iteration 30, inertia 1125230.7312804393
Iteration 31, inertia 1125145.7269572103
Iteration 32, inertia 1125070.5105915833
Iteration 33, inertia 1125006.135640854
Iteration 34, inertia 1124950.9105776069
Iteration 35, inertia 1124893.6110541513
Iteration 36, inertia 1124832.56793077
Iteration 37, inertia 1124773.9032692479
Iteration 38, inertia 1124714.4289126948
Iteration 39, inertia 1124645.5395061276
Iteration 40, inertia 1124563.66607158

In [22]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict
import os
from os import getenv

SPOTIPY_CLIENT_ID = getenv('SPOTIPY_CLIENT_ID')
SPOTIPY_SECRET_ID = getenv('SECRET_ID')

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID,
                                                           client_secret=SPOTIPY_SECRET_ID))

def find_song(name, year):
  song_data = defaultdict()
  results = sp.search(q= 'track: {} year: {}'.format(name,year), limit=1)
  if results['tracks']['items'] == []:
    return None

  results = results['tracks']['items'][0]
  track_id = results['id']
  audio_features = sp.audio_features(track_id)[0]

  song_data['name'] = [name]
  song_data['year'] = [year]
  song_data['explicit'] = [int(results['explicit'])]
  song_data['duration_ms'] = [results['duration_ms']]
  song_data['popularity'] = [results['popularity']]

  for key, value in audio_features.items():
    song_data[key] = value

  return pd.DataFrame(song_data)

In [23]:
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
import difflib

def get_song_data(song, df):

  try:
    song_data = df[(df['name'] == song['name'])].iloc[0]
    # & (df['year'] == song['year'])
    return song_data
  
  except IndexError:
    return find_song(song['name']) # , song['year']

def get_mean_vector(song_list, df):
  
  song_vectors = []

  for song in song_list:
    song_data = get_song_data(song, df)
    if song_data is None:
      print('Warning: {} does not exist in Spotify or in database'.format(song['name']))
      continue
    song_vector = song_data[number_cols].values
    song_vectors.append(song_vector)

  song_matrix = np.array(list(song_vectors))
  return np.mean(song_matrix, axis=0)

def flatten_dict_list(dict_list):
  flattened_dict = defaultdict()
  for key in dict_list[0].keys():
    flattened_dict[key] = []
  
  for dictionary in dict_list:
    for key, value in dictionary.items():
      flattened_dict[key].append(value)
  return flattened_dict

def recommend_songs(song_list, df=df, n_songs=10):
  metadata_cols = ['name', 'year', 'artists']
  song_dict = flatten_dict_list(song_list)

  song_center = get_mean_vector(song_list, df)
  scaler = song_cluster_pipeline.steps[0][1]
  scaled_data = scaler.transform(df[number_cols])
  scaled_song_center = scaler.transform(song_center.reshape(1,-1))
  distances = cdist(scaled_song_center, scaled_data, 'cosine')
  index = list(np.argsort(distances)[:, :n_songs][0])

  rec_songs = df.iloc[index]
  rec_songs = rec_songs[~rec_songs['name'].isin(song_dict['name'])]
  return rec_songs[metadata_cols].to_dict(orient='records')

In [24]:
recommend_songs([{'name': 'thriller',
                  'year': 1984}])

[{'name': "We're a Winner - Live @ Bitter End, NYC",
  'year': 1971,
  'artists': 'Curtis Mayfield'},
 {'name': 'My Life Domino - Live in Krakow 2009',
  'year': 2010,
  'artists': 'Millenium'},
 {'name': "I'm Not in Love - 2004 Remaster",
  'year': 1982,
  'artists': 'Talking Heads'},
 {'name': 'Drunken Angels - Live in Krakow 2009',
  'year': 2010,
  'artists': 'Millenium'},
 {'name': 'Inspire Me - Live at Bonn, Germany, 2005',
  'year': 2020,
  'artists': 'Paul Carrack'},
 {'name': 'Lullaby - Live at Wembley 07/89 - Remix 07/09',
  'year': 1989,
  'artists': 'The Cure'},
 {'name': 'Let Us Love - Live at Carnegie Hall, New York, NY - October 1972',
  'year': 1973,
  'artists': 'Bill Withers'},
 {'name': 'Officer', 'year': 1992, 'artists': 'The Pharcyde'},
 {'name': "Jammin' - Live At The Pavillon De Paris, 1977",
  'year': 1978,
  'artists': 'Bob Marley & The Wailers'},
 {'name': 'Another Cup of Coffee - Live at Buxton Opera House, 2004',
  'year': 2020,
  'artists': 'Paul Carrack'}]

In [27]:
user_input = input('enter a song')
user_input = user_input.strip()
input_dict = {'name': user_input[0]}
recommend_songs([input_dict])

KeyError: 'year'

[{'artists': 'Curtis Mayfield',
  'name': "We're a Winner - Live @ Bitter End, NYC",
  'year': 1971},
 {'artists': 'Millenium',
  'name': 'My Life Domino - Live in Krakow 2009',
  'year': 2010},
 {'artists': 'Talking Heads',
  'name': "I'm Not in Love - 2004 Remaster",
  'year': 1982},
 {'artists': 'Millenium',
  'name': 'Drunken Angels - Live in Krakow 2009',
  'year': 2010},
 {'artists': 'Paul Carrack',
  'name': 'Inspire Me - Live at Bonn, Germany, 2005',
  'year': 2020},
 {'artists': 'The Cure',
  'name': 'Lullaby - Live at Wembley 07/89 - Remix 07/09',
  'year': 1989},
 {'artists': 'Bill Withers',
  'name': 'Let Us Love - Live at Carnegie Hall, New York, NY - October 1972',
  'year': 1973},
 {'artists': 'The Pharcyde', 'name': 'Officer', 'year': 1992},
 {'artists': 'Bob Marley & The Wailers',
  'name': "Jammin' - Live At The Pavillon De Paris, 1977",
  'year': 1978},
 {'artists': 'Paul Carrack',
  'name': 'Another Cup of Coffee - Live at Buxton Opera House, 2004',
  'year': 2020}]